# Сверим результаты:

## Загрузим обученную модель

In [8]:
from joblib import load

# Загрузка модели из файла
model = load('models/trained_classification_model.joblib')

# Посмотреть информацию о модели
print(model)

RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=200,
                       random_state=42)


## Получим используемые метрики

In [2]:
import pandas as pd

In [3]:
features_file_path = '../autogen_studio/data/X.csv'
features_df = pd.read_csv(features_file_path)

# Load the label data
labels_file_path = '../autogen_studio/data/y.csv'
labels_df = pd.read_csv(labels_file_path)

numerical_columns = features_df.select_dtypes(include=['number']).columns
features_df[numerical_columns] = features_df[numerical_columns].fillna(features_df[numerical_columns].mean())

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Handle missing values
# Fill missing values for numerical columns with the mean
numerical_columns = features_df.select_dtypes(include=['number']).columns
features_df[numerical_columns] = features_df[numerical_columns].fillna(features_df[numerical_columns].mean())

# Fill missing values for categorical columns with the mode
for column in features_df.select_dtypes(include=['object']).columns:
    features_df[column] = features_df[column].fillna(features_df[column].mode()[0])

# Encode categorical features
label_encoders = {}
for column in features_df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    features_df[column] = label_encoders[column].fit_transform(features_df[column])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test

(        reg_time  fc_imp_chk  fc_time_chk  utmtr  mm_dma  osName  model  \
 151656     94224           0            7      1     505      10   1447   
 837875    558896           4            6      6     524      10   1447   
 152405     94701           0            7      1     556       0   1140   
 777623    517617           0            7      6     544      10   1447   
 426226    280409           0            7      0     698      10   1447   
 ...          ...         ...          ...    ...     ...     ...    ...   
 259178    168766           0            7      0     524       0    841   
 365838    239259           0            7      0     691      10   1447   
 131932     83460           0            7      0     524       0    718   
 671155    443616           0            7      6     609      10   1447   
 121958     77240           0            7      7     659       0    720   
 
         hardware  site_id  
 151656         4     9669  
 837875         4     6037  

In [12]:
from sklearn.metrics import classification_report

predictions = model.predict(X_test)
report = classification_report(y_test, predictions)

In [13]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    191426
           1       0.61      0.01      0.03       771

    accuracy                           1.00    192197
   macro avg       0.80      0.51      0.51    192197
weighted avg       0.99      1.00      0.99    192197



## Вывод
Модель показывает отличную точность при классификации объектов класса 0, но её результаты для класса 1 оставляют желать лучшего. Детальный анализ метрик выявил следующее:
- Точность для класса 0 равна 1.00, но для класса 1 всего 0.61, что означает правильное предсказание только в 61% случаев.
- Полнота для класса 0 также идеальна (1.00), но для класса 1 крайне низка - всего 0.01, указывая на обнаружение лишь 1% реальных случаев.
- F1-score для класса 0 идеальна (1.00), но для класса 1 низка (0.03), указывая на низкое качество классификации.
- Общая точность 1.00 маскирует дисбаланс классов; модель, всегда предсказывающая класс 0, также имеет высокую общую точность.
- Macro avg и Weighted avg показывают средние значения, но макро avg 0.51 говорит о слабой средней производительности модели.
В общем, модель неэффективна при работе с классом 1 и требует доработки. Возможные шаги включают перебалансировку классов, изменение порога классификации, применение техник улучшения для несбалансированных данных и анализ ошибок для коррекции предобработки данных или архитектуры модели.

### Заключение

Извините за недоразумение. Давайте добавим конкретные рекомендации к заключению:

Модель демонстрирует эффективность при работе с классом 0, но существенные недостатки в классификации класса 1. Для улучшения её производительности рекомендуется рассмотреть следующие шаги:

1. **Перебалансировка данных:**
   Учтите дисбаланс классов, увеличив количество примеров класса 1 в обучающем наборе данных.

2. **Изменение порога классификации:**
   Экспериментируйте с порогом классификации, чтобы балансировать точность и полноту в зависимости от потребностей вашей задачи.

3. **Применение техник улучшения:**
   Рассмотрите взвешивание классов, использование других метрик, таких как AUC-ROC, или применение техник обработки несбалансированных данных.

4. **Анализ ошибок:**
   Проведите дополнительный анализ ошибок, чтобы понять, почему модель плохо справляется с классом 1. Это может включать в себя коррекцию процесса предобработки данных или архитектуры модели.

Эти шаги должны помочь улучшить производительность модели, особенно в отношении класса 1, и сделать её более устойчивой к различным данным.